In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *

# Initialize Spark with connection to your cluster and HDFS
spark = SparkSession.builder \
    .appName("Amazon Reviews Analysis") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

print("Connected to Spark!")

Connected to Spark!


In [3]:
from pyspark.ml.clustering import LocalLDAModel
from pyspark.ml.feature import CountVectorizerModel
from pyspark.sql.types import ArrayType, StringType, FloatType

# Choose a category to analyze
category = "Arts"

categories = [
    "Amazon_Instant_Video",
    "Arts",
    "Automotive",
    "Baby",
    "Beauty",
    "Books",
    "Cell_Phones_&_Accessories",
    "Clothing_&_Accessories",
    "Electronics",
    "Gourmet_Foods",
    "Health",
    "Home_&_Kitchen",
    "Industrial_&_Scientific",
    "Jewelry",
    "Kindle_Store",
    "Movies_&_TV",
    "Music",
    "Musical_Instruments",
    "Office_Products",
    "Patio",
    "Pet_Supplies",
    "Shoes",
    "Software",
    "Sports_&_Outdoors",
    "Tools_&_Home_Improvement",
    "Toys_&_Games",
    "Video_Games",
    "Watches"
]

# Load the saved models
lda_model = LocalLDAModel.load(f"hdfs://namenode:9000/user/jovyan/amazon_reviews/models/lda_model_{category}")
vectorizer_model = CountVectorizerModel.load(f"hdfs://namenode:9000/user/jovyan/amazon_reviews/models/vectorizer_{category}")

# Load topic terms
topic_terms = spark.read.parquet(f"hdfs://namenode:9000/user/jovyan/amazon_reviews/results/topics_{category}")

# Display topic terms to understand what each topic represents
topic_terms.select("topic", "terms").show(truncate=False)

# Load original reviews
reviews = spark.read.json(f"hdfs://namenode:9000/user/hadoop/amazon_reviews/data/filtered_data/{category}.filtered.json")

print(f"Loaded {reviews.count()} reviews for {category}")

+-----+--------------------------------------------------------------------------------+
|topic|terms                                                                           |
+-----+--------------------------------------------------------------------------------+
|3    |[bed, great, machine, tool, use, perfect, one, shelf, clasps, good]             |
|4    |[table, chairs, one, use, bins, chalk, great, made, product, paper]             |
|8    |[hook, crochet, hooks, yarn, chiller, lenses, clay, pastel, coating, crocheting]|
|9    |[rulers, silver, use, ring, cigar, one, asin, thimble, like, work]              |
|2    |[beads, easel, colors, received, crystals, great, like, old, product, canvas]   |
|5    |[great, good, hoop, like, product, one, buy, price, machine, instructions]      |
|0    |[mat, product, cutting, use, iron, great, thimble, well, work, good]            |
|1    |[glue, serger, use, needles, used, one, machine, thread, good, great]           |
|6    |[great, use, m

In [4]:
from pyspark.ml.feature import RegexTokenizer, StopWordsRemover

# Combine review text
reviews = reviews.withColumn(
    "full_text", 
    F.concat_ws(" ", F.col("Review/Summary"), F.col("Review/Text"))
)

# Tokenize
tokenizer = RegexTokenizer(
    inputCol="full_text", 
    outputCol="words", 
    pattern="\\W+"
)
reviews_tokenized = tokenizer.transform(reviews)

# Remove stopwords
remover = StopWordsRemover(
    inputCol="words", 
    outputCol="filtered_words"
)
reviews_filtered = remover.transform(reviews_tokenized)

# Filter short words
filter_short_udf = F.udf(
    lambda words: [word for word in words if len(word) > 2],
    ArrayType(StringType())
)
reviews_filtered = reviews_filtered.withColumn(
    "filtered_words",
    filter_short_udf(F.col("filtered_words"))
)

In [6]:
# Apply the vectorizer model
reviews_vectorized = vectorizer_model.transform(reviews_filtered)

# Apply the LDA model to get topic distributions
reviews_with_topics = lda_model.transform(reviews_vectorized)

# Get the primary topic for each review
def get_primary_topic(distribution):
    return float(distribution.argmax())

get_primary_topic_udf = F.udf(get_primary_topic, FloatType())

reviews_with_topics = reviews_with_topics.withColumn(
    "primary_topic", 
    get_primary_topic_udf(F.col("topicDistribution"))
)

# Verify that topics are assigned
reviews_with_topics.select("product/productID", "Review/Score", "primary_topic").show(5)

+-----------------+------------+-------------+
|product/productID|Review/Score|primary_topic|
+-----------------+------------+-------------+
|       B000G6HRZE|         3.0|          6.0|
|       B000G6HRZE|         4.0|          6.0|
|       B000G6HRZE|         5.0|          7.0|
|       B000G6HRZE|         4.0|          7.0|
|       B000G6HRZE|         1.0|          7.0|
+-----------------+------------+-------------+
only showing top 5 rows



In [14]:
# Calculate average rating by topic
topic_ratings = reviews_with_topics.groupBy("primary_topic").agg(
    F.avg("Review/Score").alias("avg_rating"),
    F.count("*").alias("review_count"),
    F.stddev("Review/Score").alias("rating_stddev")
)

# Join with topic terms for better readability
topic_insights = topic_ratings.join(
    topic_terms.select("topic", "terms"), 
    topic_ratings.primary_topic == topic_terms.topic
)

# Show results sorted by rating
print(f"\nTOPIC RATINGS FOR {category}:")
topic_insights.select(
    "primary_topic", 
    "terms", 
    "avg_rating", 
    "review_count"
).orderBy(F.desc("avg_rating")).show(truncate=False)


TOPIC RATINGS FOR Arts:
+-------------+--------------------------------------------------------------------------------+------------------+------------+
|primary_topic|terms                                                                           |avg_rating        |review_count|
+-------------+--------------------------------------------------------------------------------+------------------+------------+
|8.0          |[hook, crochet, hooks, yarn, chiller, lenses, clay, pastel, coating, crocheting]|4.764705882352941 |51          |
|3.0          |[bed, great, machine, tool, use, perfect, one, shelf, clasps, good]             |4.388888888888889 |18          |
|0.0          |[mat, product, cutting, use, iron, great, thimble, well, work, good]            |4.262114537444934 |455         |
|6.0          |[great, use, machine, one, good, paper, product, well, like, easy]              |4.176974495424751 |15409       |
|9.0          |[rulers, silver, use, ring, cigar, one, asin, thimble, li

In [15]:
reviews = reviews.withColumn(
    "review_date", 
    F.from_unixtime(F.col("Review/Time")).cast("date")
)

# Check some sample dates to verify conversion
print("Sample of review dates:")
reviews.select("Review/Time", "review_date").show(5)

# Add year and month columns for time-based analysis
reviews = reviews.withColumn("review_year", F.year("review_date"))
reviews = reviews.withColumn("review_month", F.month("review_date"))
reviews = reviews.withColumn("year_month", F.date_format("review_date", "yyyy-MM"))

# Show the data with new date columns
print("Reviews with date columns:")
reviews.select("review_date", "review_year", "review_month", "year_month").show(5)

Sample of review dates:
+-----------+-----------+
|Review/Time|review_date|
+-----------+-----------+
| 1269043200| 2010-03-20|
| 1321833600| 2011-11-21|
| 1360886400| 2013-02-15|
| 1360627200| 2013-02-12|
| 1358899200| 2013-01-23|
+-----------+-----------+
only showing top 5 rows

Reviews with date columns:
+-----------+-----------+------------+----------+
|review_date|review_year|review_month|year_month|
+-----------+-----------+------------+----------+
| 2010-03-20|       2010|           3|   2010-03|
| 2011-11-21|       2011|          11|   2011-11|
| 2013-02-15|       2013|           2|   2013-02|
| 2013-02-12|       2013|           2|   2013-02|
| 2013-01-23|       2013|           1|   2013-01|
+-----------+-----------+------------+----------+
only showing top 5 rows



In [16]:
# Combine review text
reviews = reviews.withColumn(
    "full_text", 
    F.concat_ws(" ", F.col("Review/Summary"), F.col("Review/Text"))
)

# Tokenize
tokenizer = RegexTokenizer(
    inputCol="full_text", 
    outputCol="words", 
    pattern="\\W+"
)
reviews_tokenized = tokenizer.transform(reviews)

# Remove stopwords
remover = StopWordsRemover(
    inputCol="words", 
    outputCol="filtered_words"
)
reviews_filtered = remover.transform(reviews_tokenized)

# Filter short words
filter_short_udf = F.udf(
    lambda words: [word for word in words if len(word) > 2],
    ArrayType(StringType())
)
reviews_filtered = reviews_filtered.withColumn(
    "filtered_words",
    filter_short_udf(F.col("filtered_words"))
)

# Apply the vectorizer model
reviews_vectorized = vectorizer_model.transform(reviews_filtered)

# Apply the LDA model to get topic distributions
reviews_with_topics = lda_model.transform(reviews_vectorized)

# Get the primary topic for each review
def get_primary_topic(distribution):
    return float(distribution.argmax())

get_primary_topic_udf = F.udf(get_primary_topic, FloatType())

reviews_with_topics = reviews_with_topics.withColumn(
    "primary_topic", 
    get_primary_topic_udf(F.col("topicDistribution"))
)

# Verify that topics are assigned
print("Reviews with assigned topics:")
reviews_with_topics.select("review_date", "Review/Score", "primary_topic").show(5)


Reviews with assigned topics:
+-----------+------------+-------------+
|review_date|Review/Score|primary_topic|
+-----------+------------+-------------+
| 2010-03-20|         3.0|          6.0|
| 2011-11-21|         4.0|          6.0|
| 2013-02-15|         5.0|          7.0|
| 2013-02-12|         4.0|          7.0|
| 2013-01-23|         1.0|          7.0|
+-----------+------------+-------------+
only showing top 5 rows



In [17]:
# Calculate topic distribution by year-month
topic_time_distribution = reviews_with_topics.groupBy("year_month", "primary_topic").count()

# Show raw counts
print("Raw topic counts by time period:")
topic_time_distribution.orderBy("year_month", "primary_topic").show(10)

# Pivot to get each topic as a column
topic_time_pivot = topic_time_distribution.groupBy("year_month").pivot("primary_topic").sum("count").fillna(0)

# Sort by year_month
topic_time_pivot = topic_time_pivot.orderBy("year_month")

# Show the time series of topic prevalence
print("Topic Prevalence Over Time (year-month):")
topic_time_pivot.show(10)


Raw topic counts by time period:
+----------+-------------+-----+
|year_month|primary_topic|count|
+----------+-------------+-----+
|      NULL|          0.0|    1|
|   1999-08|          6.0|    1|
|   2000-05|          6.0|    1|
|   2000-08|          6.0|    2|
|   2000-10|          0.0|    2|
|   2000-10|          6.0|    1|
|   2000-11|          8.0|    1|
|   2001-01|          6.0|    1|
|   2001-02|          6.0|    1|
|   2001-03|          6.0|    1|
+----------+-------------+-----+
only showing top 10 rows

Topic Prevalence Over Time (year-month):
+----------+---+---+---+---+---+---+---+---+---+---+
|year_month|0.0|1.0|2.0|3.0|4.0|5.0|6.0|7.0|8.0|9.0|
+----------+---+---+---+---+---+---+---+---+---+---+
|      NULL|  1|  0|  0|  0|  0|  0|  0|  0|  0|  0|
|   1999-08|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|
|   2000-05|  0|  0|  0|  0|  0|  0|  1|  0|  0|  0|
|   2000-08|  0|  0|  0|  0|  0|  0|  2|  0|  0|  0|
|   2000-10|  2|  0|  0|  0|  0|  0|  1|  0|  0|  0|
|   2000-11|  

In [18]:
# Calculate average rating by topic and time period
topic_rating_time = reviews_with_topics.groupBy("year_month", "primary_topic") \
    .agg(
        F.avg("Review/Score").alias("avg_rating"),
        F.count("*").alias("review_count")
    )

# Filter to ensure statistical significance (at least 3 reviews per topic per time period)
significant_ratings = topic_rating_time.filter(F.col("review_count") >= 3)

# Join with topic terms for better readability
significant_ratings = significant_ratings.join(
    topic_terms.select("topic", "terms"),
    significant_ratings.primary_topic == topic_terms.topic
)

# Show ratings over time
print("Topic Ratings Over Time (with at least 3 reviews):")
significant_ratings.select(
    "year_month", 
    "primary_topic", 
    "terms", 
    "avg_rating", 
    "review_count"
).orderBy("year_month", "primary_topic").show(10, truncate=False)

Topic Ratings Over Time (with at least 3 reviews):
+----------+-------------+------------------------------------------------------------------+------------------+------------+
|year_month|primary_topic|terms                                                             |avg_rating        |review_count|
+----------+-------------+------------------------------------------------------------------+------------------+------------+
|2003-01   |6.0          |[great, use, machine, one, good, paper, product, well, like, easy]|4.5               |4           |
|2003-11   |6.0          |[great, use, machine, one, good, paper, product, well, like, easy]|2.6               |5           |
|2003-12   |6.0          |[great, use, machine, one, good, paper, product, well, like, easy]|2.6666666666666665|3           |
|2004-04   |6.0          |[great, use, machine, one, good, paper, product, well, like, easy]|4.5               |4           |
|2004-08   |6.0          |[great, use, machine, one, good, paper, p

In [24]:
# Group by month (regardless of year) to see if certain topics are seasonal
monthly_topic_dist = reviews_with_topics.groupBy("review_month", "primary_topic").count()

# Calculate total reviews by month
monthly_totals = reviews_with_topics.groupBy("review_month").count().withColumnRenamed("count", "monthly_total")

# Join to calculate percentages
monthly_topic_dist = monthly_topic_dist.join(monthly_totals, "review_month")
monthly_topic_dist = monthly_topic_dist.withColumn(
    "percentage", 
    (F.col("count") / F.col("monthly_total")) * 100
)

# Join with topic terms
monthly_topic_dist = monthly_topic_dist.join(
    topic_terms.select("topic", "terms"),
    monthly_topic_dist.primary_topic == topic_terms.topic
)

# Create a window spec to rank topics within each month by percentage
from pyspark.sql.window import Window
window_spec = Window.partitionBy("review_month").orderBy(F.desc("percentage"))

# Add rank column
monthly_topic_ranked = monthly_topic_dist.withColumn("rank", F.rank().over(window_spec))

# Filter to keep only top 3 topics per month
top3_monthly_topics = monthly_topic_ranked.filter(F.col("rank") <= 3)

# Show monthly patterns (top 3 for each month)
print("Top 3 Topics by Month (potential seasonality):")
top3_monthly_topics.select(
    "review_month", 
    "primary_topic", 
    "terms", 
    "percentage", 
    "count",
    "rank"
).orderBy("review_month", "rank").show(36, truncate=False)

Top 3 Topics by Month (potential seasonality):
+------------+-------------+---------------------------------------------------------------------+------------------+-----+----+
|review_month|primary_topic|terms                                                                |percentage        |count|rank|
+------------+-------------+---------------------------------------------------------------------+------------------+-----+----+
|1           |6.0          |[great, use, machine, one, good, paper, product, well, like, easy]   |85.29411764705883 |2755 |1   |
|1           |1.0          |[glue, serger, use, needles, used, one, machine, thread, good, great]|9.845201238390093 |318  |2   |
|1           |0.0          |[mat, product, cutting, use, iron, great, thimble, well, work, good] |2.1362229102167185|69   |3   |
|2           |6.0          |[great, use, machine, one, good, paper, product, well, like, easy]   |85.47979797979798 |2031 |1   |
|2           |1.0          |[glue, serger, use, ne

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.ml.clustering import LocalLDAModel
from pyspark.ml.feature import CountVectorizerModel, RegexTokenizer, StopWordsRemover
from pyspark.sql.window import Window

# Initialize Spark
spark = SparkSession.builder \
    .appName("Amazon Reviews Multi-Category Analysis") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

print("Connected to Spark!")

# List of categories to analyze
categories = [
    "Amazon_Instant_Video",
    "Arts",
    "Automotive",
    "Baby",
    "Beauty",
    "Cell_Phones_&_Accessories",
    "Clothing_&_Accessories",
    "Electronics",
    "Gourmet_Foods",
    "Health",
    "Home_&_Kitchen",
    "Industrial_&_Scientific",
    "Jewelry",
    "Movies_&_TV",
    "Music",
    "Musical_Instruments",
    "Office_Products",
    "Patio",
    "Pet_Supplies",
    "Shoes",
    "Software",
    "Sports_&_Outdoors",
    "Tools_&_Home_Improvement",
    "Toys_&_Games",
    "Video_Games",
    "Watches"
]

# Function to process a single category
def analyze_category_top_topics(category):
    try:
        print(f"\n\n======= ANALYZING CATEGORY: {category} =======\n")
        
        # Load topic terms
        topic_terms = spark.read.parquet(f"hdfs://namenode:9000/user/jovyan/amazon_reviews/results/topics_{category}")
        
        # Display topic terms
        print(f"Topic terms for category: {category}")
        topic_terms.select("topic", "terms").show(truncate=False)
        
        # Load original reviews
        reviews = spark.read.json(f"hdfs://namenode:9000/user/hadoop/amazon_reviews/data/filtered_data/{category}.filtered.json")
        print(f"Loaded {reviews.count()} reviews for {category}")
        
        # Load models
        lda_model = LocalLDAModel.load(f"hdfs://namenode:9000/user/jovyan/amazon_reviews/models/lda_model_{category}")
        vectorizer_model = CountVectorizerModel.load(f"hdfs://namenode:9000/user/jovyan/amazon_reviews/models/vectorizer_{category}")
        
        # Convert Unix timestamp to date
        reviews = reviews.withColumn(
            "review_date", 
            F.from_unixtime(F.col("Review/Time")).cast("date")
        )
        
        # Add time-related columns
        reviews = reviews.withColumn("review_year", F.year("review_date"))
        reviews = reviews.withColumn("review_month", F.month("review_date"))
        reviews = reviews.withColumn("year_month", F.date_format("review_date", "yyyy-MM"))
        
        # Combine review text
        reviews = reviews.withColumn(
            "full_text", 
            F.concat_ws(" ", F.col("Review/Summary"), F.col("Review/Text"))
        )
        
        # Tokenize
        tokenizer = RegexTokenizer(
            inputCol="full_text", 
            outputCol="words", 
            pattern="\\W+"
        )
        reviews_tokenized = tokenizer.transform(reviews)
        
        # Remove stopwords
        remover = StopWordsRemover(
            inputCol="words", 
            outputCol="filtered_words"
        )
        reviews_filtered = remover.transform(reviews_tokenized)
        
        # Filter short words
        filter_short_udf = F.udf(
            lambda words: [word for word in words if len(word) > 2],
            ArrayType(StringType())
        )
        reviews_filtered = reviews_filtered.withColumn(
            "filtered_words",
            filter_short_udf(F.col("filtered_words"))
        )
        
        # Apply the vectorizer model
        reviews_vectorized = vectorizer_model.transform(reviews_filtered)
        
        # Apply the LDA model to get topic distributions
        reviews_with_topics = lda_model.transform(reviews_vectorized)
        
        # Get the primary topic for each review
        def get_primary_topic(distribution):
            return float(distribution.argmax())
        
        get_primary_topic_udf = F.udf(get_primary_topic, FloatType())
        
        reviews_with_topics = reviews_with_topics.withColumn(
            "primary_topic", 
            get_primary_topic_udf(F.col("topicDistribution"))
        )
        
        # Group by month (regardless of year) to see if certain topics are seasonal
        monthly_topic_dist = reviews_with_topics.groupBy("review_month", "primary_topic").count()
        
        # Calculate total reviews by month
        monthly_totals = reviews_with_topics.groupBy("review_month").count().withColumnRenamed("count", "monthly_total")
        
        # Join to calculate percentages
        monthly_topic_dist = monthly_topic_dist.join(monthly_totals, "review_month")
        monthly_topic_dist = monthly_topic_dist.withColumn(
            "percentage", 
            (F.col("count") / F.col("monthly_total")) * 100
        )
        
        # Join with topic terms
        monthly_topic_dist = monthly_topic_dist.join(
            topic_terms.select("topic", "terms"),
            monthly_topic_dist.primary_topic == topic_terms.topic
        )
        
        # Create a window spec to rank topics within each month by percentage
        window_spec = Window.partitionBy("review_month").orderBy(F.desc("percentage"))
        
        # Add rank column
        monthly_topic_ranked = monthly_topic_dist.withColumn("rank", F.rank().over(window_spec))
        
        # Filter to keep only top 5 topics per month
        top5_monthly_topics = monthly_topic_ranked.filter(F.col("rank") <= 5)
        
        # Show monthly patterns (top 5 for each month)
        print(f"Top 5 Topics by Month for {category}:")
        top5_monthly_topics.select(
            "review_month", 
            "primary_topic", 
            "terms", 
            "percentage", 
            "count",
            "rank"
        ).orderBy("review_month", "rank").show(60, truncate=False)
        
        # Calculate overall top 5 topics for the category
        overall_topic_dist = reviews_with_topics.groupBy("primary_topic").count()
        total_reviews = reviews_with_topics.count()
        
        overall_topic_dist = overall_topic_dist.withColumn(
            "percentage", 
            (F.col("count") / total_reviews) * 100
        )
        
        # Join with topic terms
        overall_topic_dist = overall_topic_dist.join(
            topic_terms.select("topic", "terms"),
            overall_topic_dist.primary_topic == topic_terms.topic
        )
        
        # Show overall top topics
        print(f"Overall Top Topics for {category}:")
        overall_topic_dist.select(
            "primary_topic", 
            "terms", 
            "percentage", 
            "count"
        ).orderBy(F.desc("percentage")).show(5, truncate=False)
        
        # Save category-specific results
        output_path = f"hdfs://namenode:9000/user/jovyan/amazon_reviews/results/{category}_top5_monthly_topics"
        print(f"Saving results for {category} to {output_path}")
        top5_monthly_topics.write.mode("overwrite").parquet(output_path)
        
        # Also save overall topic distribution
        overall_output_path = f"hdfs://namenode:9000/user/jovyan/amazon_reviews/results/{category}_overall_topics"
        overall_topic_dist.write.mode("overwrite").parquet(overall_output_path)
        
        print(f"Results for {category} saved successfully!")
        
        return {
            "category": category,
            "success": True
        }
        
    except Exception as e:
        print(f"Error processing category {category}: {str(e)}")
        return {
            "category": category,
            "success": False,
            "error": str(e)
        }

# Process each category
results = []

for category in categories:
    result = analyze_category_top_topics(category)
    results.append(result)

# Print summary
print("\nProcessing Summary:")
successful = 0
failed = 0

for result in results:
    status = "Success" if result["success"] else f"Failed: {result.get('error', 'Unknown error')}"
    print(f"{result['category']}: {status}")
    
    if result["success"]:
        successful += 1
    else:
        failed += 1

print(f"\nTotal categories processed: {len(results)}")
print(f"Successfully processed: {successful}")
print(f"Failed to process: {failed}")

Connected to Spark!


======= ANALYZING CATEGORY: Amazon_Instant_Video =======

Topic terms for category: Amazon_Instant_Video
+-----+----------------------------------------------------------------------+
|topic|terms                                                                 |
+-----+----------------------------------------------------------------------+
|3    |[dvd, movie, one, good, amazon, like, video, time, ray, quality]      |
|4    |[movie, one, film, good, like, life, man, time, also, gay]            |
|8    |[movie, time, get, like, one, money, film, waste, dvd, western]       |
|9    |[film, one, story, people, world, documentary, life, man, movie, also]|
|7    |[movie, good, see, one, documentary, great, people, food, film, watch]|
|2    |[christmas, film, smith, movie, one, allen, great, like, case, woody] |
|6    |[movie, like, one, good, film, really, great, watch, see, story]      |
|0    |[great, show, love, movie, season, one, series, film, good, watch]    |
|5  